In [1]:
import triton
import triton.language as tl

@triton.jit
def reverse_kernel(
    input_ptr,
    N,
    BLOCK_SIZE: tl.constexpr
):
    input_ptr = input_ptr.to(tl.pointer_type(tl.float32))
    pid=tl.program_id(0)
    blockstart=pid*BLOCK_SIZE
    offsets=blockstart+tl.arange(0,BLOCK_SIZE)
    mask=offsets<N//2
    offsets_last=N-offsets-1

    val1=tl.load(input_ptr+offsets,mask=mask)
    val2=tl.load(input_ptr+offsets_last,mask=mask)
    tl.store(input_ptr+offsets,val2,mask=mask)
    tl.store(input_ptr+offsets_last,val1,mask=mask)
    

def solve(input_ptr: int, N: int):
    BLOCK_SIZE = 1024
    n_blocks = triton.cdiv(N // 2, BLOCK_SIZE)
    grid = (n_blocks,)
    
    reverse_kernel[grid](
        input_ptr,
        N,
        BLOCK_SIZE
    ) 